In [ ]:
import pandas as pd
import nltk
import string

# Baixa os recursos necessários do NLTK
nltk.download('punkt')        # Tokenizador pré-treinado
nltk.download('stopwords')    # Lista de stopwords (palavras irrelevantes) em vários idiomas
nltk.download('wordnet')      # Base de dados semântica usada na lematização (em inglês)

# Importa componentes do NLTK
from nltk.corpus import stopwords  # Lista de palavras irrelevantes para remoção
from nltk.stem import SnowballStemmer, RSLPStemmer, PorterStemmer, WordNetLemmatizer  # Stemmers e lematizador
from nltk.tokenize import TweetTokenizer  # Tokenizador ideal para textos informais (como tweets, emojis, hashtags)

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Carregando DF
df = pd.read_csv('database/aa_dataset-tickets-multi-lang-5-2-50-version.csv')
df.head(2)

In [ ]:
df.dtypes

In [ ]:
# Verificando idiomas do DF
print(df['language'].unique())

# Verificando categorias dos chamados
print(df['type'].unique())


In [ ]:
# Filtrando os emaisl apenas na lingua inglesa
df_en = df[df['language'] == "en"]

# Removendo colunas não utilizadas
df_en = df_en.drop(columns= ["subject", "answer", "queue", "priority", "language", "version", "tag_1", "tag_2", "tag_3", "tag_4", "tag_5", "tag_6", "tag_7", "tag_8"])

In [ ]:
# Verificando se os dados necessário estão no df
df_en.isnull().sum()

# Pré-processamento do texto

### Tokenização

Dividir o texto em unidades menores que podem ser facilmente processados ​​por modelos de PNL.

In [ ]:
def tokenizador(data):
    # Inicializar a classe tokenizer
    tokenizer = TweetTokenizer()
    tokenized_text = tokenizer.tokenize(data)
    
    return tokenized_text

In [ ]:
df_en['tokenized_body'] = df_en['body'].apply(tokenizador)

In [ ]:
df_en.head(5)

### Remoção de stopwords

São consideradas como tendo pouco valor semântico ou são insignificantes para a tarefa

Cada linguagem, tem sua lista de stop words. Como a língua dos textos que estamos pré-processando é inglês, vamos lidar com elas

In [ ]:
# Importando a lista de stop words em inglês da biblioteca NLTK
stopwords_english = stopwords.words('english') 

# Exemplos
print('Stop words - inglês\n')
print(stopwords_english)

print('\nPontuação\n')
print(string.punctuation)

In [ ]:
def stopwords(data):

    text_cleaned = []

    for word in data: # analisa cada palavra da lista de tokens
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove pontuação
            text_cleaned.append(word)

    return text_cleaned

In [ ]:
df_en['stopword_removed'] = df_en['tokenized_body'].apply(stopwords)

In [ ]:
df_en.head(5)

### Stemming 

Processo de reduzir a forma flexionada de uma palavra ao “radical” ou forma raiz, também conhecido como “lema” em linguística.

In [ ]:
def stem(data):
    # Iniciliza a classe stemming
    stemmer = PorterStemmer()

    # Cria lista vazia para armazenar os stems
    text_stems = []

    for word in data:
        stem_word = stemmer.stem(word)
        text_stems.append(stem_word)
        
    return text_stems

In [ ]:
df_en['stemmed_text'] = df_en['stopword_removed'].apply(stem)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Junta todos os textos em uma única string
texto = ' '.join(df_en['stemmed_text'].astype(str))

# Cria a nuvem de palavras
nuvem = WordCloud(
    background_color='white',
    stopwords=stopwords_english,
    width=800,
    height=400,
    colormap='viridis'
).generate(texto)

# Exibe a nuvem
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.imshow(nuvem, interpolation='bilinear')
plt.axis('off')
plt.title("Nuvem de Palavras")
plt.show()

## Vetorização

### TF- IDF
TF-IDF é o produto da Frequência do Termo e da Frequência Inversa do Documento. 

In [ ]:
# Converta para string, caso necessário
corpus = df_en['body'].astype(str)

# Inicializa o vetorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')  # Pode incluir mais params como ngram_range, max_features, etc.
X_tfidf = vectorizer.fit_transform(corpus)

# Supondo que sua coluna 'body' já esteja pré-processada e unida como texto
corpus = df_en['body'].astype(str)

# Criar o vetor TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)  # Limite de features, pode ajustar
tfidf_matrix = vectorizer.fit_transform(corpus)

# tfidf_matrix é uma matriz esparsa (documentos x palavras)
print("Shape:", tfidf_matrix.shape)

#  transformar em um DataFrame:
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df_tfidf